# Create chm_fp collection in genra_db

In [1]:
# %load ../env.py
%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 

#Set environment variables


# Set up the local source files
TOP = '/'.join(os.getcwd().split('/')[:-2])+'/'
#TOP = "/share/home/ishah/ipynb/pb/HTTR_WF/"

LIB = TOP+'lib'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

os.environ['PYTHONPATH']=LIB


DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

if not os.path.exists(DAT_DIR): os.mkdir(DAT_DIR)
if not os.path.exists(FIG_DIR): os.mkdir(FIG_DIR)


from db.mongo import *

DB = openMongo(db='genra_dev_v4')


Populating the interactive namespace from numpy and matplotlib


In [2]:
DB.collection_names()
DB.compounds.find_one().keys()

[u'smiles',
 u'dsstox_sid',
 u'chemspider_id',
 u'created_at',
 u'iupac',
 u'updated_at',
 u'pubchem_cid',
 u'gsid',
 u'synonyms',
 u'dsstox_cid',
 u'mol_weight',
 u'inchi_key',
 u'_id',
 u'casrn',
 u'name']

In [6]:
DB.compounds.find_one(dict(name='Bisphenol A'))['dsstox_sid']

u'DTXSID7020182'

In [4]:
from chm.mkfp import *

In [10]:
makeFP('DTXSID7020182', col_comp=DB.compounds,col_chm_fp=DB.chm_fp,save=True)

In [11]:
for C in DB.compounds.find():
    makeFP()

{u'_id': ObjectId('5a3ab42b9fb21e52969ab186'),
 u'casrn': u'80-05-7',
 u'dsstox_cid': u'DTXCID30182',
 u'dsstox_sid': u'DTXSID7020182',
 u'httr': {u'ds': [u'httr_0',
   u'httr_1',
   u'httr_2',
   u'httr_372',
   u'httr_373',
   u'httr_374',
   u'httr_375',
   u'httr_380',
   u'httr_381',
   u'httr_382',
   u'httr_383',
   u'httr_1488',
   u'httr_1489',
   u'httr_1490',
   u'httr_1892',
   u'httr_1893',
   u'httr_1894',
   u'httr_2008',
   u'httr_2009',
   u'httr_2010'],
  u'n': 20},
 u'mrgn': {u'ds': [u'mrgn_8',
   u'mrgn_114',
   u'mrgn_189',
   u'mrgn_231',
   u'mrgn_392',
   u'mrgn_532',
   u'mrgn_679',
   u'mrgn_745',
   u'mrgn_807',
   u'mrgn_1011',
   u'mrgn_1057',
   u'mrgn_1075',
   u'mrgn_1076',
   u'mrgn_1313',
   u'mrgn_1380',
   u'mrgn_1417',
   u'mrgn_1602',
   u'mrgn_1665',
   u'mrgn_1750',
   u'mrgn_1778',
   u'mrgn_1873'],
  u'n': 21},
 u'name': u'Bisphenol A'}

In [3]:
for i in ['dsstox_sid']: DB.chm_fp.create_index(i)

In [7]:
DB.chm_fp.count()

998

# Serial FP generation


In [ ]:
import datetime
from chm.mkfp import *

N = DB.compounds.count() - DB.chm_fp.count()
print "Start ... ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",N
    
k=0
for C in DB.compounds.find(no_cursor_timeout=True):
    k+=1
    makeFP(C['dsstox_sid'], col_comp=DB.compounds,col_chm_fp=DB.chm_fp,
           save=True,replace=False)
        
    if k%1000==0:
        print k,'/',N,datetime.datetime.now().strftime("%H:%M:%S")
        
print "Ended ... ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",N

Start ...  2017-12-20 17:08:47 tasks =  636960
1000 / 636960 17:08:49
2000 / 636960 17:08:52
3000 / 636960 17:08:56
4000 / 636960 17:09:00
5000 / 636960 17:09:04
6000 / 636960 17:09:09
7000 / 636960 17:09:15
8000 / 636960 17:09:21
9000 / 636960 17:09:27
10000 / 636960 17:09:34
11000 / 636960 17:09:42
12000 / 636960 17:09:50
13000 / 636960 17:09:59
14000 / 636960 17:10:08
15000 / 636960 17:10:18
16000 / 636960 17:10:28
17000 / 636960 17:10:39
18000 / 636960 17:10:51
19000 / 636960 17:11:03
20000 / 636960 17:11:15
21000 / 636960 17:11:28
22000 / 636960 17:11:41


# Parallel FP generation

In [2]:
import ipyparallel as PP

RC=PP.Client(profile='chiron_pm')
d_view=RC[:]
lb_view = RC.load_balanced_view()
lb_view.block = True
x=file("../env.py",'r').read()
d_view.execute(x)

<AsyncResult: execute>

In [3]:
from chm.mkfp import *

In [7]:
d_view.execute("from chm.mkfp import *")
d_view.execute("DB = openMongo(db='genra_dev_v4')")

<AsyncResult: execute>

In [19]:
DB.chm_fp.find_one().keys()

[u'mrgn', u'dsstox_sid', u'name', u'httr', u'casrn', u'dsstox_cid', u'_id']

In [2]:
#DB.chm_fp.delete_many({})
for i in ['name','mrgn.ds','mrgn.n','httr.n','httr.ds','casrn',
          'dsstox_cid','dsstox_sid']:
    print i
    DB.chm_fp.create_index(i)


name
mrgn.ds
mrgn.n
httr.n
httr.ds
casrn
dsstox_cid
dsstox_sid


In [ ]:
from threading import Thread
SID = []

def procCurs(curs,res):
    for C in curs:
        makeFP(C['dsstox_sid'], col_comp=DB.compounds,col_chm_fp=DB.chm_fp,
               save=True,replace=False)
        
        
Curs = DB.compounds.parallel_scan(50)
TH0  = [Thread(target=procCurs,args=(curs,SID)) for curs in Curs]
for th in TH0: th.start()
for th in TH0: th.join()


## How many chemicals left to do

In [ ]:
DONE = DB.chm_fp.distinct('dsstox_sid')
WORK = []

for C in DB.compounds.find():
    if DB.chm_fp.find(dict(dsstox_sid=C['dsstox_sid'])).count()>0: 
        continue
    else:
        WORK.append(C['dsstox_sid'])
#list(set(DB.compounds.distinct('dsstox_sid')).difference(DONE))
len(WORK)

## Run in parallel

In [ ]:
import datetime
from chm.mkfp import *

            
print "Start ... ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",len(WORK)

lb_view.map(lambda dtxsid: makeFP(dtxsid, col_comp=DB.compounds,col_chm_fp=DB.chm_fp,save=True),WORK)


print "End ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "tasks = ",len(WORK)



Start ...  2017-12-20 14:21:00 tasks =  750464


In [ ]:
DB.chm_fp.find_one()